In [72]:
import json
import pandas as pd
import requests
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100
pd.set_option('display.max_colwidth', -1)

/var/folders/cg/b2_z5vjn52x7nyrtvtmwqsqh0000gp/T/ipykernel_27627/2703078761.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [73]:
def getDataframe(filename):
    arr = []
    with open('./data/' + filename) as out:
        for line in out:
            j_content = json.loads(line)
            arr.append(j_content)
    df = pd.DataFrame(arr)
    return df

In [74]:
organizations = getDataframe('organizations.json')

In [75]:
arr = []
for index, row in organizations.iterrows():
    odict = {}
    odict['geop_organization_id'] = dict(row['_id'])['$oid']
    odict['geop_organization_name'] = row['name']
    arr.append(odict)
organizations = pd.DataFrame(arr)

In [76]:
organizations['dgv_organization_id'] = organizations['geop_organization_id']

In [77]:
dgv_catalog_orgas = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/b7bbfedc-2448-4135-a6c7-104548d396e7',sep=';')

In [78]:
dgv_catalog_orgas = dgv_catalog_orgas[['id','name']]

In [79]:
dgv_catalog_orgas = dgv_catalog_orgas.rename(columns={'id': 'dgv_organization_id','name':'dgv_organization_name'})

In [80]:
organizations = organizations.drop_duplicates(keep="first")
dgv_catalog_orgas = dgv_catalog_orgas.drop_duplicates(keep="first")

In [81]:
organizations.columns

Index(['geop_organization_id', 'geop_organization_name',
       'dgv_organization_id'],
      dtype='object')

In [82]:
dgv_catalog_orgas.dgv_organization_id.nunique()

3209

In [83]:
final_orgas = pd.merge(organizations,dgv_catalog_orgas,on='dgv_organization_id',how='left')

In [84]:
geoc = pd.read_excel('./data/Liste_catalogues_DCAT.xlsx')

In [85]:
geoc = geoc.rename(columns={'Producteurs': 'geoc_organization_name', 
                            'uuid': 'geoc_organization_uuid', 
                            'url de moissonnage ': 'geoc_organization_url'
})

In [86]:
dff = final_orgas
dff['geoc_organization_name'] = None
dff['geoc_organization_uuid'] = None
dff['geoc_organization_url'] = None

In [87]:
def removeCommonWords(org):
    org = str(org)
    org = org.replace(' de ',' ').replace(' un ',' ').replace(' à ','').replace(' l\'',' ').replace(' d\'', ' ')
    org = org.replace(' des ',' ').replace(' pour ', ' ').replace(' les ', ' ').replace('(','').replace(')','')
    org = org.replace(' - ',' ').replace(',','').replace(' du ', ' ').replace('à','a').replace('é','e')
    org = org.replace('ê','e').replace('è','e').replace('\'',' ')
    return org.lower()

In [88]:
arr = []
cpt = 0
for index, row in dff.iterrows():
    for index2, row2 in geoc.iterrows():
        if(row['geop_organization_name'] == row['geop_organization_name']):
            if removeCommonWords(row['geop_organization_name']) in removeCommonWords(row2['geoc_organization_name']):
                row['geoc_organization_name'] = row2['geoc_organization_name']
                row['geoc_organization_uuid'] = row2['geoc_organization_uuid']
                row['geoc_organization_url'] = row2['geoc_organization_url']
            elif(row['dgv_organization_name'] == row['dgv_organization_name']):
                if removeCommonWords(row['dgv_organization_name']) in removeCommonWords(row2['geoc_organization_name']):
                    row['geoc_organization_name'] = row2['geoc_organization_name']
                    row['geoc_organization_uuid'] = row2['geoc_organization_uuid']
                    row['geoc_organization_url'] = row2['geoc_organization_url']

    arr.append(row)

In [89]:
df_final = pd.DataFrame(arr)

In [90]:
df_final.shape

(296, 7)

In [93]:
df_final[df_final['geoc_organization_uuid'].notna()].shape

(56, 7)

In [92]:
df_final.to_csv('mapping_geop_dgv_geoc.csv',index=False)